In [13]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

/aiffel/aiffel/aimmo/mmdetection


In [14]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

0.13.5


In [15]:
# # 완디비 로그인
wandb.login(key= '')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [16]:
wandb.init(project="r-cnn", entity="aimmo")

In [17]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_texts')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [ ]:
config_file = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg = Config.fromfile(config_file)

cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'


cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_sunday_test.txt'
cfg.data.test.img_prefix = 'test/image'

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg.work_dir = 'middle_fulldata_fullsize'

#cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
#cfg.evaluation.save_best='auto'
cfg.data.train.pipeline[2].img_scale=[(960,960)]#['Resize']
cfg.train_pipeline[2].img_scale=[(960,960)]
cfg.test_pipeline[1].img_scale=[(960,960)]
cfg.data.val.pipeline[1].img_scale = [(960,960)]
cfg.data.test.pipeline[1].img_scale = [(960,960)]
cfg.runner.max_epochs = 15

cfg.data.samples_per_gpu = 12 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 0 # Worker to pre-fetch data for each single GPU
print(cfg.pretty_text)

In [7]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'r-cnn', 'entity' : 'aimmo', 'name' : 'fulldata_fullsize'},)] 

In [8]:
[build_dataset(cfg.data.test)]

/aiffel/aiffel/aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


[
 Aimmo Train dataset with number of images 797, and instance counts: 
 +----------+-------+-----------+-------+----------+-------+----------------+-------+---------------+-------+
 | category | count | category  | count | category | count | category       | count | category      | count |
 +----------+-------+-----------+-------+----------+-------+----------------+-------+---------------+-------+
 | 0 [car]  | 3482  | 1 [truck] | 1143  | 2 [bus]  | 696   | 3 [pedestrian] | 1030  | -1 background | 0     |
 +----------+-------+-----------+-------+----------+-------+----------------+-------+---------------+-------+]

In [9]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-08 11:26:31,859 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-08 11:26:34,135 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-12-08 11:26:35,551 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([16]).
20

2022-12-08 11:31:17,726 - mmdet - INFO - Epoch [1][100/1742]	lr: 3.976e-03, eta: 19:56:03, time: 2.757, data_time: 1.427, memory: 10037, loss_rpn_cls: 0.0393, loss_rpn_bbox: 0.0433, loss_cls: 0.5259, acc: 81.6624, loss_bbox: 0.5051, loss: 1.1136
2022-12-08 11:35:53,144 - mmdet - INFO - Epoch [1][200/1742]	lr: 7.972e-03, eta: 19:50:53, time: 2.754, data_time: 1.401, memory: 10037, loss_rpn_cls: 0.0318, loss_rpn_bbox: 0.0439, loss_cls: 0.2446, acc: 91.5996, loss_bbox: 0.3328, loss: 0.6531
2022-12-08 11:40:28,390 - mmdet - INFO - Epoch [1][300/1742]	lr: 1.197e-02, eta: 19:45:50, time: 2.752, data_time: 1.400, memory: 10037, loss_rpn_cls: 0.0273, loss_rpn_bbox: 0.0391, loss_cls: 0.2200, acc: 92.3740, loss_bbox: 0.2978, loss: 0.5841
2022-12-08 11:45:03,192 - mmdet - INFO - Epoch [1][400/1742]	lr: 1.596e-02, eta: 19:40:33, time: 2.748, data_time: 1.394, memory: 10037, loss_rpn_cls: 0.0276, loss_rpn_bbox: 0.0399, loss_cls: 0.2173, acc: 92.3079, loss_bbox: 0.3011, loss: 0.5859
2022-12-08 11:49

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 4.9 task/s, elapsed: 476s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-08 12:55:36,837 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 25544 | 0.940  | 0.902 |
| truck      | 3084  | 14021 | 0.892  | 0.765 |
| bus        | 1786  | 8875  | 0.914  | 0.825 |
| pedestrian | 3063  | 9411  | 0.766  | 0.622 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.779 |
+------------+-------+-------+--------+-------+
2022-12-08 12:55:36,963 - mmdet - INFO - Epoch(val) [1][2323]	AP50: 0.7790, mAP: 0.7788
2022-12-08 13:00:18,908 - mmdet - INFO - Epoch [2][100/1742]	lr: 2.000e-02, eta: 18:22:50, time: 2.814, data_time: 1.455, memory: 10037, loss_rpn_cls: 0.0183, loss_rpn_bbox: 0.0366, loss_cls: 0.1686, acc: 93.8564, loss_bbox: 0.2553, loss: 0.4788
2022-12-08 13:04:58,934 - mmdet - INFO - Epoch [2][200/1742]	lr: 2.000e-02, eta: 18:19:52, time: 2.800, data_time: 1.444, memory: 10037, loss_rp

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 4.9 task/s, elapsed: 472s, ETA:     0s
---------------iou_thr: 0.5---------------


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 462s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-09 08:10:26,900 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 19275 | 0.965  | 0.949 |
| truck      | 3084  | 6031  | 0.921  | 0.878 |
| bus        | 1786  | 2987  | 0.934  | 0.909 |
| pedestrian | 3063  | 6357  | 0.845  | 0.771 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.877 |
+------------+-------+-------+--------+-------+
2022-12-09 08:10:27,012 - mmdet - INFO - Epoch(val) [14][2323]	AP50: 0.8770, mAP: 0.8766
2022-12-09 08:15:01,062 - mmdet - INFO - Epoch [15][100/1742]	lr: 2.000e-04, eta: 1:14:29, time: 2.736, data_time: 1.435, memory: 10037, loss_rpn_cls: 0.0051, loss_rpn_bbox: 0.0207, loss_cls: 0.0823, acc: 96.8273, loss_bbox: 0.1528, loss: 0.2609
2022-12-09 08:19:35,576 - mmdet - INFO - Epoch [15][200/1742]	lr: 2.000e-04, eta: 1:09:57, time: 2.745, data_time: 1.428, memory: 10037, loss_r

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.1 task/s, elapsed: 458s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-09 09:37:56,974 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 19069 | 0.964  | 0.948 |
| truck      | 3084  | 6022  | 0.922  | 0.878 |
| bus        | 1786  | 2936  | 0.935  | 0.910 |
| pedestrian | 3063  | 6374  | 0.847  | 0.772 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.877 |
+------------+-------+-------+--------+-------+
2022-12-09 09:37:57,083 - mmdet - INFO - Epoch(val) [15][2323]	AP50: 0.8770, mAP: 0.8772


learning_rate,▇█████████████████████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇███████████████
train/loss,█▆▅▅▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,█▆▅▅▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_cls,█▆▅▅▅▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▆▆▅▅▅▅▄▄▄▄▃▄▃▃▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_cls,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/AP50,▁▄▅▆▆▆▆▆███████
val/mAP,▁▄▅▅▆▆▆▆███████
learning_rate,0.0002
